In [2]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Input, BatchNormalization, ReLU, MaxPool2D, UpSampling2D,GlobalAveragePooling2D,Dense,Dropout



  


In [ ]:
class Identity_Block(keras.layers.Layer):

  def __init__(self,filters = None,f = None):

    super(Identity_Block,self).__init__()
    f1,f2,f3 = filters

    self.conv1 = Conv2D(f1,(1,1),padding = 'valid')
    self.bn1 = BatchNormalization()
    self.act1  = ReLU()

    self.conv2 = Conv2D(f2,(f,f),padding = 'same')
    self.bn2 = BatchNormalization()
    self.act2  = ReLU()

    
    self.conv3 = Conv2D(f3,(1,1),padding = 'valid')
    self.bn3 = BatchNormalization()
    self.act3  = ReLU()
  
  def call(self,x):

    hx = x

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.act1(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.act2(x)

    x = self.conv3(x)
    x = self.bn3(x)

    y = x + hx
    y = self.act3(y)

    return y


In [3]:

class ConvBNAct(keras.layers.Layer):
  
  def __init__(self,f1=None,s = None,k = None):

    super(ConvBNAct,self).__init__()

    self.conv1 = Conv2D(f1,k,s,padding='valid')
    self.bn1 = BatchNormalization()
    self.act1 = ReLU()
    self.maxpool = MaxPool2D(pool_size = 2)

  def call(self,x):

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.act1(x)
    x = self.maxpool(x)

    return x

In [25]:

class ClassifierLayer(keras.layers.Layer):

  def __init__(self,num_class=None,dropout =None,units = None):

    super(ClassifierLayer,self).__init__()

    self.GlobalAvgPool = GlobalAveragePooling2D()
    self.dense1 = Dense(units,activation='relu')
    self.dropout = Dropout(rate = dropout)
    self.dense2 = Dense(num_class)

  def call(self,x):

    x = self.GlobalAvgPool(x)

    x = self.dense1(x)
    x = self.dropout(x)
    x = self.dense2(x)


    return x


In [44]:
class Convolution_Block(keras.layers.Layer):

  def __init__(self,filters = None,f = None):

    super(Convolution_Block,self).__init__()

    f1,f2,f3 = filters


    self.conv1 = Conv2D(f1,(1,1),padding = 'valid',strides=2)
    self.bn1 = BatchNormalization()
    self.act1  = ReLU()

    self.conv2 = Conv2D(f2,(f,f),padding = 'same')
    self.bn2 = BatchNormalization()
    self.act2  = ReLU()

    
    self.conv3 = Conv2D(f3,(1,1),padding = 'valid')
    self.bn3 = BatchNormalization()


    self.skip_conv3 = Conv2D(f3,(1,1),padding = 'same',strides=2)
    self.skip_bn3 = BatchNormalization()
    self.act3  = ReLU()

  def call(self,x):

    hx = x 

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.act1(x)

    x = self.conv2(x)
    x = self.bn2(x)
    x = self.act2(x)

    x = self.conv3(x)
    x = self.bn3(x)

    skip_x = self.skip_conv3(hx)
    skip_x = self.skip_bn3(skip_x)

    y = (x+skip_x)
    y = self.act3(y)

    return x

In [45]:



def resnet50(input_shape=input_shape,num_class=num_class,dropout=dropout,units=units):
  
  inp = Input(shape = input_shape)
  
  x = ConvBNAct(f1=64,s = 2,k = 5)(inp)

  x = Convolution_Block(f = 3,filters = (16,16,32))(x)
  x = Identity_Block(filters = (16,16,32),f=3)(x)
  x = Identity_Block(filters = (16,16,32),f=3)(x)
  # x = MaxPool2D(2)(x)

  x = Convolution_Block(f = 3,filters = (32,32,64))(x)
  x = Identity_Block(filters = (32,32,64),f=3)(x)
  x = Identity_Block(filters = (32,32,64),f=3)(x)
  x = Identity_Block(filters = (32,32,64),f=3)(x)
  # x = MaxPool2D(2)(x)



  x = Convolution_Block(f = 3,filters = (64,64,128))(x)
  x = Identity_Block(filters = (64,64,128),f=3)(x)
  x = Identity_Block(filters = (64,64,128),f=3)(x)
  x = Identity_Block(filters = (64,64,128),f=3)(x)
  x = Identity_Block(filters = (64,64,128),f=3)(x)
  x = Identity_Block(filters = (64,64,128),f=3)(x)
  # x = MaxPool2D(2)(x)



  x = Convolution_Block(f = 3,filters = (128,128,256))(x)
  x = Identity_Block(filters = (128,128,256),f=3)(x)
  x = Identity_Block(filters = (128,128,256),f=3)(x)
  x = Identity_Block(filters = (128,128,256),f=3)(x)
  x = Identity_Block(filters = (128,128,256),f=3)(x)
  x = Identity_Block(filters = (128,128,256),f=3)(x)
  x = Identity_Block(filters = (128,128,256),f=3)(x)
  x = Identity_Block(filters = (128,128,256),f=3)(x)
  # x = MaxPool2D(2)(x)


  y = ClassifierLayer(num_class = num_class,dropout = dropout,units=units)(x)

  model = keras.models.Model(inputs = inp,outputs = y )

  return model



In [46]:

input_shape=(224,224,3)
num_class=10
dropout=0.4
units=512
model = resnet50(input_shape=input_shape,num_class=num_class,dropout=dropout,units=units)
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv_bn_act_16 (ConvBNAct)   (None, 55, 55, 64)        5120      
_________________________________________________________________
convolution__block_60 (Convo (None, 28, 28, 32)        6368      
_________________________________________________________________
identity__block_255 (Identit (None, 28, 28, 32)        3648      
_________________________________________________________________
identity__block_256 (Identit (None, 28, 28, 32)        3648      
_________________________________________________________________
convolution__block_61 (Convo (None, 14, 14, 64)        15296     
_________________________________________________________________
identity__block_257 (Identit (None, 14, 14, 64)        139